# Day 15: Warehouse Woes

In [ ]:
import numpy as np
from time import sleep, time
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

with open('data/2024-15-example.txt', 'r') as f:
    test_data = f.readlines()
with open('data/2024-15.txt', 'r') as f:
    data = f.readlines()

test_data = [x.strip() for x in test_data]
data = [x.strip() for x in data]

print('Example data:')
print('\n'.join(test_data))

In [2]:
data_map = {
    '.': 0,
    '@': 1,
    'O': 2,
    '[': 3,
    ']': 4,
    '#': 6
}
movement_map = {
    '<': np.array([0, -1]),
    '>': np.array([0, 1]),
    '^': np.array([-1, 0]),
    'v': np.array([1, 0])
}

def setup_data(data_input: list, large_out: bool = False):
    split_idx = data_input.index('')
    warehouse = data_input[:split_idx]
    movements = ''.join(data_input[split_idx+1:])
    initial_pos = None
    
    # setup warehouse
    h, w = len(warehouse), len(warehouse[0])
    h, w = h*2 if large_out else h, w*2 if large_out else w
    w_arr = np.zeros((h, w), dtype=int)
    
    for i, row in enumerate(warehouse):
        
        # convert to large if necessary
        if large_out:
            row = row.replace('#', '##')
            row = row.replace('O', '[]')
            row = row.replace('.', '..')
            row = row.replace('@', '@.')
        
        for j, item in enumerate(row):
            w_arr[i, j] = data_map[item]
            if item == '@':
                initial_pos = np.array([i, j])
    return w_arr, movements, initial_pos


def compute_GPS(warehouse_map: np.ndarray, map_char: str = 'O') -> int:
    weight_array = np.array([100, 1])
    box_pos = np.argwhere(warehouse_map == data_map[map_char])
    return (box_pos * weight_array).sum()

## Part 1

In [3]:
def update_pos_1(warehouse_map: np.ndarray, pos: np.ndarray, mov_char: str):
    new_pos = pos + movement_map[mov_char]
    new_pos_val = warehouse_map[*new_pos]
    
    # do nothing if next position is a wall
    if new_pos_val == data_map['#']:
        return warehouse_map, pos

    # if next position is a box see if there is an empty space behind it
    if new_pos_val == data_map['O']:
        new_box_pos = new_pos + movement_map[mov_char]
        while True:
            # if next position is a wall, we cannot move the box, thus stand still
            if warehouse_map[*new_box_pos] == data_map['#']:
                return warehouse_map, pos
            # if next position is empty, move the box there
            elif warehouse_map[*new_box_pos] == data_map['.']:
                warehouse_map[*new_box_pos] = data_map['O']
                break
                
            # check next position
            new_box_pos += movement_map[mov_char]
    
    # if next position is a big box
    if new_pos_val in [data_map['['], data_map[']']]:
        # get left and right position of the box
        bpl = new_pos if new_pos_val == data_map['['] else new_pos - np.array([0, 1])
        bpr = bpl + np.array([0, 1])
        
        while True:
            pass
    
    # update robot position
    warehouse_map[*pos] = data_map['.']
    warehouse_map[*new_pos] = data_map['@']
    return warehouse_map, new_pos


def compute_movements_1(warehouse_map: np.ndarray, movements: str, pos: np.ndarray, do_plot: bool = False):
    if do_plot:
        fig, ax = plt.subplots(figsize=(6, 6))
        mat = ax.matshow(warehouse_map, cmap=plt.cm.Blues)
        plt.axis('off')
        plt.title('Initial Warehouse')

    for movement in mov:
        warehouse_map, pos = update_pos_1(warehouse_map, pos, movement)
        if do_plot:
            plt.title(f'Move {movement}')
            mat.set_data(warehouse_map)
            clear_output(wait=True)
            display(fig)
    
    return warehouse_map

In [ ]:
w_arr, mov, pos = setup_data(data)
w_arr = compute_movements_1(w_arr, mov, pos, do_plot=False)
print('Part 1:', compute_GPS(w_arr, 'O'))

## Part 2

In [5]:
def check_movement_dfs(warehouse_map: np.ndarray, pos: np.ndarray, mov_char: str):
    # check if current position is a wall or empty space
    if warehouse_map[*pos] == data_map['#']:
        return False
    if warehouse_map[*pos] == data_map['.']:
        return True
    
    # now check movement of boxes
    new_pos = pos + movement_map[mov_char]
    
    # horizontal movement
    if mov_char in ['<', '>']:
        if check_movement_dfs(warehouse_map, new_pos, mov_char):
            return True
    # vertical movement
    else:
        if warehouse_map[*pos] == data_map['[']:
            extra_check = np.array([0, 1]) 
        elif warehouse_map[*pos] == data_map[']']:
            extra_check = np.array([0, -1])
        else:
            return False
        if check_movement_dfs(warehouse_map, new_pos, mov_char) and \
            check_movement_dfs(warehouse_map, new_pos + extra_check, mov_char):
            return True
    return False


def update_boxes_dfs(warehouse_map: np.ndarray, pos: np.ndarray, mov_char: str):
    # stop if wall or empty space is reached
    if warehouse_map[*pos] == data_map['#']:
        return False
    if warehouse_map[*pos] == data_map['.']:
        return True
    
    # now check movement of boxes
    new_pos = pos + movement_map[mov_char]

    # horizontal movement
    if mov_char in ['<', '>']:
        if update_boxes_dfs(warehouse_map, new_pos, mov_char):
            warehouse_map[*new_pos] = warehouse_map[*pos]
            warehouse_map[*pos] = data_map['.']
            return True
    else:
        if warehouse_map[*pos] == data_map['[']:
            extra_check = np.array([0, 1]) 
        elif warehouse_map[*pos] == data_map[']']:
            extra_check = np.array([0, -1])
        else:
            return False

        new_pos_2 = new_pos + extra_check
        pos_2 = pos + extra_check
        if update_boxes_dfs(warehouse_map, new_pos, mov_char) and update_boxes_dfs(warehouse_map, new_pos_2, mov_char):
            # update new position (both left and right)
            warehouse_map[*new_pos] = warehouse_map[*pos]
            warehouse_map[*new_pos_2] = warehouse_map[*pos_2]
            
            # update old position (both left and right)
            warehouse_map[*pos] = data_map['.']
            warehouse_map[*pos_2] = data_map['.']
            return True
    return False


def compute_movements_2(warehouse_map: np.ndarray, movements: str, pos: np.ndarray, do_plot: bool = False):
    if do_plot:
        fig, ax = plt.subplots(figsize=(6, 3))
        mat = ax.matshow(warehouse_map, cmap=plt.cm.Blues)
        plt.axis('off')
        plt.title('Initial Warehouse')

    for movement in mov:
        new_pos = pos + movement_map[movement]
        if not check_movement_dfs(warehouse_map, new_pos, movement):
            continue

        # update box positions
        update_boxes_dfs(warehouse_map, new_pos, movement)
        
        # update robot position
        warehouse_map[*pos] = data_map['.']
        warehouse_map[*new_pos] = data_map['@']
        pos = new_pos
        
        if do_plot:
            plt.title(f'Move {movement}')
            mat.set_data(warehouse_map)
            clear_output(wait=True)
            display(fig)
    
    return warehouse_map

In [ ]:
w_arr, mov, pos = setup_data(data, large_out=True)
w_arr = compute_movements_2(w_arr, mov, pos, do_plot=False)
print('Part 2:', compute_GPS(w_arr, '['))